In [ ]:
spark

In [ ]:
storage_account_name = 'azuresynaptraining'
storage_account_access_key = '=='
spark.conf.set('.' + storage_account_name + '.blob.core.windows.net', storage_account_access_key)
blob_container = 'task'
filePath = f"" + blob_container + "@" + storage_account_name + "."
zomato_df = spark.read.format("csv").load(filePath, inferSchema = True, header = True)

In [ ]:
zomato_df.show()

+-------+----------+--------------+----------------+--------+-----+----------+
|OrderID|CustomerID|RestaurantName|        DishName|Quantity|Price| OrderDate|
+-------+----------+--------------+----------------+--------+-----+----------+
|      1|       183|     Spice Hub|    Paneer Tikka|       3|  322|2024-06-04|
|      2|       186|   Sushi World|    Cheeseburger|       5|  466|2024-06-23|
|      3|       127|   Pasta Place|    Cheeseburger|       5|  260|2024-06-08|
|      4|       145|  Pizza Palace|  Mango Smoothie|       2|  382|2024-06-26|
|      5|       144|  Pizza Palace| Pepperoni Pizza|       1|  154|2024-06-26|
|      6|       110|  Pizza Palace|    Paneer Tikka|       1|  411|2024-06-18|
|      7|       156|   Pasta Place|    Cheeseburger|       1|  110|2024-06-05|
|      8|       126|   Sushi World|  Veggie Delight|       5|  494|2024-06-05|
|      9|       108|  Burger Haven| Pepperoni Pizza|       3|  474|2024-06-02|
|     10|       119|   Pasta Place| Pepperoni Pizza|

In [ ]:
zomato_df.createOrReplaceTempView('zomato_db')

In [ ]:
from pyspark.sql.functions import unix_timestamp, current_timestamp, window, col
from pyspark.sql.types import TimestampType

zomato_stream = spark.readStream\
    .format("csv") \
    .schema(zomato_df.schema) \
    .option("header", "true") \
    .load(filePath)\
    .withColumn('timestamp', current_timestamp().cast(TimestampType()))

zomato_stream = zomato_stream.withWatermark("timestamp", "5 minutes")


zomato_stream.createOrReplaceTempView('zomato_db')

result_df = spark.sql("""
        SELECT RestaurantName, AVG(CustomerOrderCount) AS AvgOrdersPerCustomer,timestamp
        FROM (
            SELECT RestaurantName, CustomerID, COUNT(OrderID) AS CustomerOrderCount,timestamp
            FROM zomato_db
            GROUP BY RestaurantName, CustomerID, timestamp
        ) AS CustomerOrderCounts
        GROUP BY RestaurantName, timestamp
        ORDER BY AvgOrdersPerCustomer DESC
        """)

In [ ]:
config = {
  "spark.cosmos.accountEndpoint": "https://cosmostestdbtraining.documents.azure.com:443/",
  "spark.cosmos.accountKey": "==",
  "spark.cosmos.database": "test_db",
  "spark.cosmos.container": "container"
}

# Configure Catalog Api    
spark.conf.set("spark.sql.catalog.cosmosCatalog", "com.azure.cosmos.spark.CosmosCatalog")
spark.conf.set("spark.sql.catalog.cosmosCatalog.spark.cosmos.accountEndpoint", config["spark.cosmos.accountEndpoint"])
spark.conf.set("spark.sql.catalog.cosmosCatalog.spark.cosmos.accountKey", config["spark.cosmos.accountKey"])

def write_to_cosmos(batch_df, batch_id):
    batch_df.write \
        .format("cosmos.oltp") \
        .options(**config) \
        .mode("append") \
        .save()

# Write the streaming data to Cosmos DB using foreachBatch
streaming_query = result_df.writeStream \
    .foreachBatch(write_to_cosmos) \
    .outputMode("append") \
    .option("checkpointLocation", "/Volumes/azuredatabricks/default/checkpoint2") \
    .start()

# Wait for the streaming query to finish
streaming_query.awaitTermination()

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-2640412931704828>, line 25
     14     batch_df.write \
     15         .format("cosmos.oltp") \
     16         .options(**config) \
     17         .mode("append") \
     18         .save()
     20 # Write the streaming data to Cosmos DB using foreachBatch
     21 streaming_query = result_df.writeStream \
     22     .foreachBatch(write_to_cosmos) \
     23     .outputMode("complete") \
     24     .option("checkpointLocation", "/Volumes/azuredatabricks/default/checkpoint2") \
---> 25     .start()
     27 # Wait for the streaming query to finish
     28 streaming_query.awaitTermination()

File /databricks/spark/python/pyspark/sql/streaming/readwriter.py:1637, in DataStreamWriter.start(self, path, format, outputMode, partitionBy, queryName, **options)
   1635     self.queryName(queryName)
   1636 if path is None:
-> 1637 

In [ ]:
streaming_query = result_df.writeStream \
    .foreachBatch(write_to_cosmos) \
    .outputMode("append") \
    .option("checkpointLocation", "") \
    .start()

streaming_query.awaitTermination()

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-4254577650536249>, line 4
      1 import time
      2 streaming_query = result_df.writeStream \
      3     .foreachBatch(write_to_cosmos) \
----> 4     .start()
      5 time.sleep(3)
      9 streaming_query.awaitTermination()

File /databricks/spark/python/pyspark/sql/streaming/readwriter.py:1637, in DataStreamWriter.start(self, path, format, outputMode, partitionBy, queryName, **options)
   1635     self.queryName(queryName)
   1636 if path is None:
-> 1637     return self._sq(self._jwrite.start())
   1638 else:
   1639     return self._sq(self._jwrite.start(path))

File /databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py:1355, in JavaMember.__call__(self, *args)
   1349 command = proto.CALL_COMMAND_NAME +\
   1350     self.command_header +\
   1351     args_command +\
   1352     proto.END_COMMAND_PA